In [20]:
## load libraries
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# LangChain core imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import (
    RunnablePassthrough, 
 
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# LangChain specific imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Load environment variables
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [2]:
sample_documents = [
    Document(
        page_content="""
        Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
        """,
        metadata={"source": "AI Introduction", "page": 1, "topic": "AI"}
    ),
    Document(
        page_content="""
        Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.
        """,
        metadata={"source": "ML Basics", "page": 1, "topic": "ML"}
    ),
    Document(
        page_content="""
        Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning has revolutionized computer vision, NLP, and speech recognition.
        """,
        metadata={"source": "Deep Learning", "page": 1, "topic": "DL"}
    ),
    Document(
        page_content="""
        Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
        Applications include chatbots, translation, sentiment analysis, and text summarization.
        """,
        metadata={"source": "NLP Overview", "page": 1, "topic": "NLP"}
    )
]

print(sample_documents)

[Document(metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}, page_content='\n        Artificial Intelligence (AI) is the simulation of human intelligence in machines.\n        These systems are designed to think like humans and mimic their actions.\n        AI can be categorized into narrow AI and general AI.\n        '), Document(metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='\n        Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.\n        '), Document(metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='\n        Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolu

In [3]:
## text splitting
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=[" "]
)

## split the documents into chunks
chunks = text_splitter.split_documents(sample_documents)
print(chunks[0])
print(chunks[1])

page_content='Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.' metadata={'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}
page_content='Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised, unsupervised, and reinforcement learning.' metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}


In [4]:
print(f"Created {len(chunks)} chunks from {len(sample_documents)} documents")
print("\nExample chunk:")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Created 4 chunks from 4 documents

Example chunk:
Content: Artificial Intelligence (AI) is the simulation of human intelligence in machines.
        These systems are designed to think like humans and mimic their actions.
        AI can be categorized into narrow AI and general AI.
Metadata: {'source': 'AI Introduction', 'page': 1, 'topic': 'AI'}


In [6]:
# Initialize OpenAI embeddings with the latest model

embeddings=OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536
)


In [7]:
texts=["AI","MAchine learning","Deep Learning","Neural Network"]
batch_embeddings=embeddings.embed_documents(texts)

In [8]:
### Compare Embedding using cosine similarity

def compare_embeddings(text1:str,text2:str):
    """Compare semantic simialrity of 2 texts usign embeddings"""

    emb1=np.array(embeddings.embed_query(text1))
    emb2=np.array(embeddings.embed_query(text2))

    ## Calculate the simialrity score

    similarity=np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity

In [9]:
# Test semantic similarity
print("\nSemantic Similarity Examples:")
print(f"'AI' vs 'Artificial Intelligence': {compare_embeddings('AI', 'Artificial Intelligence'):.3f}")


Semantic Similarity Examples:
'AI' vs 'Artificial Intelligence': 0.563


In [10]:
### Create FAISS Vector Store
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print(f"Vector store created with {vectorstore.index.ntotal} vectors")

Vector store created with 4 vectors


In [12]:
## Save vector tore for later use
vectorstore.save_local("data/faiss_index")
print("Vector store saved to 'faiss_index' directory")

Vector store saved to 'faiss_index' directory


In [14]:
## load vector store
loaded_vectorstore=FAISS.load_local(
    "data/faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print(f"Loaded vector store contains {loaded_vectorstore.index.ntotal} vectors")

Loaded vector store contains 4 vectors


In [15]:
## Similarity Search 
query="What is deep learning"

results=vectorstore.similarity_search(query,k=3)
print(results)

[Document(id='eabe28f0-b020-491c-a922-929164cd2923', metadata={'source': 'Deep Learning', 'page': 1, 'topic': 'DL'}, page_content='Deep Learning is a subset of machine learning based on artificial neural networks.\n        It uses multiple layers to progressively extract higher-level features from raw input.\n        Deep learning has revolutionized computer vision, NLP, and speech recognition.'), Document(id='9520cb83-74cc-406b-a364-278236ba05d0', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.'), Document(id='e2995591-6557-44a1-a644-fbf13ae85574', metadata={'source': 'NLP Overview', 'page': 1, 'topic': 'NLP'}, page_content='Natural Language Processing (NLP) is a branch of AI that helps computers understand human lang

In [16]:
print(f"Query: {query}\n")
print("Top 3 similar chunks:")
for i, doc in enumerate(results):
    print(f"\n{i+1}. Source: {doc.metadata['source']}")
    print(f"   Content: {doc.page_content[:200]}...")

Query: What is deep learning

Top 3 similar chunks:

1. Source: Deep Learning
   Content: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses multiple layers to progressively extract higher-level features from raw input.
        Deep learning ...

2. Source: ML Basics
   Content: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being explicitly programmed, ML algorithms find patterns in data.
        Common types include supervised...

3. Source: NLP Overview
   Content: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
        It combines computational linguistics with machine learning and deep learning models.
      ...


In [17]:
### Similarity Search with score
results_with_scores=vectorstore.similarity_search_with_score(query,k=3)

print("\n\nSimilarity search with scores:")
for doc, score in results_with_scores:
    print(f"\nScore: {score:.3f}")
    print(f"Source: {doc.metadata['source']}")
    print(f"Content preview: {doc.page_content[:100]}...")



Similarity search with scores:

Score: 0.556
Source: Deep Learning
Content preview: Deep Learning is a subset of machine learning based on artificial neural networks.
        It uses m...

Score: 1.208
Source: ML Basics
Content preview: Machine Learning is a subset of AI that enables systems to learn from data.
        Instead of being...

Score: 1.274
Source: NLP Overview
Content preview: Natural Language Processing (NLP) is a branch of AI that helps computers understand human language.
...


In [18]:
### Search with metadata filtering
filter_dict={"topic":"ML"}
filtered_results=vectorstore.similarity_search(
    query,
    k=3,
    filter=filter_dict
)
print(filtered_results)

[Document(id='9520cb83-74cc-406b-a364-278236ba05d0', metadata={'source': 'ML Basics', 'page': 1, 'topic': 'ML'}, page_content='Machine Learning is a subset of AI that enables systems to learn from data.\n        Instead of being explicitly programmed, ML algorithms find patterns in data.\n        Common types include supervised, unsupervised, and reinforcement learning.')]


In [21]:
## LLM GROQ LLM
from langchain.chat_models import init_chat_model

llm=init_chat_model(model="groq:gemma2-9b-it")

In [22]:
llm.invoke("Hi")

AIMessage(content='Hi! 👋 \n\nWhat can I do for you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 10, 'total_tokens': 26, 'completion_time': 0.029090909, 'prompt_time': 0.001188679, 'queue_time': 0.24913354, 'total_time': 0.030279588}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--71df07c3-e7dc-4a83-b8d4-295802721006-0', usage_metadata={'input_tokens': 10, 'output_tokens': 16, 'total_tokens': 26})

In [23]:
# 1. Simple RAG Chain with LCEL
simple_prompt = ChatPromptTemplate.from_template("""Answer the question based only on the following context:
Context: {context}

Question: {question}

Answer:""")

In [24]:
## Basic retriever
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [25]:
from typing import List
# Format documents for the prompt
def format_docs(docs: List[Document]) -> str:
    """Format documents for insertion into prompt"""
    formatted = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1} (Source: {source}):\n{doc.page_content}")
    return "\n\n".join(formatted)

In [26]:
simple_rag_chain=(
    {"context":retriever | format_docs,"question":RunnablePassthrough() }
    | simple_prompt
    | llm
    |StrOutputParser()

)

In [27]:
### Conversational RAg Chain

conversational_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer questions."),
    ("placeholder", "{chat_history}"),
    ("human", "Context: {context}\n\nQuestion: {input}"),
])

In [28]:
def create_conversational_rag():
    """Create a conversational RAG chain with memory"""
    return (
        RunnablePassthrough.assign(
            context=lambda x: format_docs(retriever.invoke(x["input"]))
        )
        | conversational_prompt
        | llm
        | StrOutputParser()
    )

conversational_rag = create_conversational_rag()

In [29]:
### streaming RAG chain
streaming_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | simple_prompt
    | llm
)

print("Modern RAG chains created successfully!")
print("Available chains:")
print("- simple_rag_chain: Basic Q&A")
print("- conversational_rag: Maintains conversation history")
print("- streaming_rag_chain: Supports token streaming")

Modern RAG chains created successfully!
Available chains:
- simple_rag_chain: Basic Q&A
- conversational_rag: Maintains conversation history
- streaming_rag_chain: Supports token streaming


In [30]:
# Test function for different chain types
def test_rag_chains(question: str):
    """Test all RAG chain variants"""
    print(f"Question: {question}")
    print("=" * 80)
    
    # 1. Simple RAG
    print("\n1. Simple RAG Chain:")
    answer = simple_rag_chain.invoke(question)
    print(f"Answer: {answer}")

    print("\n2. Streaming RAG:")
    print("Answer: ", end="", flush=True)
    for chunk in streaming_rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print()

In [31]:
test_rag_chains("What is the difference between AI and machine learning")

Question: What is the difference between AI and machine learning

1. Simple RAG Chain:
Answer: AI is the broader concept of machines simulating human intelligence, while machine learning is a subset of AI that enables systems to learn from data without explicit programming.  


2. Streaming RAG:
Answer: AI is the broad concept of machines simulating human intelligence, while machine learning is a subset of AI that focuses on enabling systems to learn from data without explicit programming. 



In [32]:
# Test with multiple questions
test_questions = [
    "What is the difference between AI and Machine Learning?",
    "Explain deep learning in simple terms",
    "How does NLP work?"
]

for question in test_questions:
    print("\n" + "=" * 80 + "\n")
    test_rag_chains(question)



Question: What is the difference between AI and Machine Learning?

1. Simple RAG Chain:
Answer: Artificial Intelligence (AI) is the broader concept of machines simulating human intelligence. Machine Learning (ML) is a subset of AI that focuses on enabling systems to learn from data without explicit programming. 


2. Streaming RAG:
Answer: Artificial Intelligence (AI) is the broad concept of machines performing tasks that typically require human intelligence. Machine Learning (ML) is a subset of AI where systems learn from data rather than being explicitly programmed.  



Question: Explain deep learning in simple terms

1. Simple RAG Chain:
Answer: Deep learning is a type of machine learning that uses artificial networks with many layers to learn complex patterns from data. Think of it like teaching a computer to see and understand the world by showing it lots of examples, similar to how humans learn. 


2. Streaming RAG:
Answer: Deep learning is a type of machine learning that uses

In [33]:
## Conversational example
print("\n3. Conversational RAG Example:")
chat_history = []

# First question
q1 = "What is machine learning?"
a1 = conversational_rag.invoke({
    "input": q1,
    "chat_history": chat_history
})

print(f"Q1: {q1}")
print(f"A1: {a1}")


3. Conversational RAG Example:
Q1: What is machine learning?
A1: According to Document 1, Machine Learning is a subset of Artificial Intelligence that allows systems to learn from data. 

Instead of being directly programmed, Machine Learning algorithms identify patterns within data. 



In [34]:
# Update history
chat_history.extend([
    HumanMessage(content=q1),
    AIMessage(content=a1)
])

In [35]:
# Follow-up question
q2 = "How is it different from traditional programming?"
a2 = conversational_rag.invoke({
    "input": q2,
    "chat_history": chat_history
})
print(f"\nQ2: {q2}")
print(f"A2: {a2}")


Q2: How is it different from traditional programming?
A2: According to Document 2, traditional programming involves explicitly writing instructions for a system to follow. 

In contrast, Machine Learning algorithms learn patterns from data instead of being directly programmed.  They identify these patterns and use them to make predictions or decisions. 

